 # Creating Model for Chatbot

Import necessary libraries

In [6]:
import nltk
nltk.download('punkt')# training dataset1
nltk.download('wordnet')# training dataset2
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /home/chandu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/chandu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Initializing Chatbot Training

"intents.json" — abunch of JavaScript objects that lists different tags that correspond to different types of word patterns

Creating two file for training model, 
 "classes.pkl" — a list of different types of classes of responses ||
 "words.pkl" — a list of different words that could be used for pattern recognition

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


Building the Deep Learning Model

In [3]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)

print("model created")

W0619 16:36:01.972824 140293936101184 deprecation_wrapper.py:119] From /home/chandu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0619 16:36:01.993056 140293936101184 deprecation_wrapper.py:119] From /home/chandu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0619 16:36:01.997419 140293936101184 deprecation_wrapper.py:119] From /home/chandu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0619 16:36:02.027855 140293936101184 deprecation_wrapper.py:119] From /home/chandu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_defa

Training data created


W0619 16:36:02.162044 140293936101184 deprecation_wrapper.py:119] From /home/chandu/.local/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0619 16:36:02.171370 140293936101184 deprecation_wrapper.py:119] From /home/chandu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0619 16:36:02.266733 140293936101184 deprecation.py:323] From /home/chandu/.local/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/200
47/47 [==============================] - 0s 7ms/step - loss: 2.2359 - acc: 0.0638
Epoch 2/200
47/47 [==============================] - 0s 462us/step - loss: 2.1770 - acc: 0.1915
Epoch 3/200
47/47 [==============================] - 0s 437us/step - loss: 2.1140 - acc: 0.2128
Epoch 4/200
47/47 [==============================] - 0s 492us/step - loss: 1.9652 - acc: 0.2766
Epoch 5/200
47/47 [==============================] - 0s 432us/step - loss: 1.8986 - acc: 0.4255
Epoch 6/200
47/47 [==============================] - 0s 592us/step - loss: 1.7161 - acc: 0.5106
Epoch 7/200
47/47 [==============================] - 0s 354us/step - loss: 1.6026 - acc: 0.5319
Epoch 8/200
47/47 [==============================] - 0s 537us/step - loss: 1.3919 - acc: 0.6596
Epoch 9/200
47/47 [==============================] - 0s 484us/step - loss: 1.2041 - acc: 0.7234
Epoch 10/200
47/47 [==============================] - 0s 568us/step - loss: 1.2836 - acc: 0.6170
Epoch 11/200
47/47 [=====================

47/47 [==============================] - 0s 299us/step - loss: 0.0355 - acc: 1.0000
Epoch 86/200
47/47 [==============================] - 0s 319us/step - loss: 0.0530 - acc: 1.0000
Epoch 87/200
47/47 [==============================] - 0s 316us/step - loss: 0.0264 - acc: 1.0000
Epoch 88/200
47/47 [==============================] - 0s 273us/step - loss: 0.0483 - acc: 0.9787
Epoch 89/200
47/47 [==============================] - 0s 280us/step - loss: 0.0219 - acc: 1.0000
Epoch 90/200
47/47 [==============================] - 0s 345us/step - loss: 0.0988 - acc: 0.9574
Epoch 91/200
47/47 [==============================] - 0s 269us/step - loss: 0.0438 - acc: 0.9787
Epoch 92/200
47/47 [==============================] - 0s 284us/step - loss: 0.0360 - acc: 1.0000
Epoch 93/200
47/47 [==============================] - 0s 283us/step - loss: 0.0403 - acc: 1.0000
Epoch 94/200
47/47 [==============================] - 0s 275us/step - loss: 0.0090 - acc: 1.0000
Epoch 95/200
47/47 [=======================

47/47 [==============================] - 0s 342us/step - loss: 0.0166 - acc: 1.0000
Epoch 169/200
47/47 [==============================] - 0s 288us/step - loss: 0.0139 - acc: 1.0000
Epoch 170/200
47/47 [==============================] - 0s 296us/step - loss: 0.0338 - acc: 0.9787
Epoch 171/200
47/47 [==============================] - 0s 338us/step - loss: 0.0205 - acc: 1.0000
Epoch 172/200
47/47 [==============================] - 0s 295us/step - loss: 0.0367 - acc: 1.0000
Epoch 173/200
47/47 [==============================] - 0s 271us/step - loss: 0.0143 - acc: 1.0000
Epoch 174/200
47/47 [==============================] - 0s 290us/step - loss: 0.0132 - acc: 1.0000
Epoch 175/200
47/47 [==============================] - 0s 319us/step - loss: 0.0316 - acc: 1.0000
Epoch 176/200
47/47 [==============================] - 0s 332us/step - loss: 0.0174 - acc: 1.0000
Epoch 177/200
47/47 [==============================] - 0s 292us/step - loss: 0.0257 - acc: 1.0000
Epoch 178/200
47/47 [=============

# Building Chatbot GUI

In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

Creating GUI with tkinter

In [5]:
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set


SendButton = Button(base, font=("Orange",12,'bold'), text="Send!", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")


scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()